### Collective Burden for Sequencing Documents

In [1]:
import requests
from collections import Counter
import random as randomlib

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import networkx as nx

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
'''
Knowledge Graph
'''
kg_path = "../graph_query/graphs/weighted_knowledge_graph.gpickle"
kg = nx.read_gpickle(kg_path)
kg_labels = [str(x) for x in list(kg.nodes())[1:]]
n_labels = len(kg_labels)

In [3]:
def get_queries_based_on_node(node_label):
    node = kg.node[node_label]

    if("NodeType" not in node): 
        kg.node[node_label]["NodeType"] = "ConceptNode"
        l =  list(kg.neighbors(node_label))
        return l
    node_type = node["NodeType"]
    if(node_type == "TopicNode" or node_type == "ConceptNode"):
        return list(kg.neighbors(node_label))
    elif(node_type == "SubConceptNode"):
        return [node_label]
    else:
        pass

'''
Returns a list of queries depending on the type of the
node closest to the query.
args - query(str)
returns [] of str
'''
def query_formulator(query, label):
    queries = []
    children_neighbours = get_queries_based_on_node(label)
    queries = [label]
    for child in children_neighbours:
        queries.append(child)
    return list(set(queries))


In [4]:
'''
Get content from a given set of URLs.
'''
def get_content(url):
    es_order = []
    f = open(url, 'r')
    l = f.readlines()
    docs = {}
    index = {}
    
    counter = 0
    for url in l:
        try:
            docs[url] = requests.get(url).content
            index[url] = counter
            es_order.append(url)
            counter += 1
        except:
            continue
    return docs, index, es_order

In [5]:
'''
Term Frequency Array for a particular document.
'''
def get_tfd(content):
    word_count_dict = Counter(w for w in kg_labels 
                              if w.lower() in content.lower())
    common = word_count_dict.most_common()
    
    frequency_arr = [0]*len(kg_labels)
    
    for common_word in common:
        common_word_index = kg_labels.index(common_word[0])
        frequency_arr[common_word_index] = common_word[1]
    return frequency_arr

In [6]:
'''
Building word_data a document (rows) by term frequency (columns) matrix.
'''
def get_matrices(content, index):
    tfd_data = {}
    for url, cont in content.items():
        tfd_data[url] = get_tfd(cont)

    tfd_arr = []
    for key in index.keys():
        tfd_arr.append(key.replace("\n", ""))

    word_data = {'TFD':tfd_arr}

    for label in kg_labels:
        word_data[label] = [None]*len(index)

    for url, words_in_doc in tfd_data.items():
        url_index = index[url]
        for i in range(0, n_labels, 1):
            word = kg_labels[i]
            word_data[word][url_index] = words_in_doc[i]

    '''
    (DTF)^T(DTF) = Coocurence Matrix
    '''
    document_term_frequency = pd.DataFrame(word_data).set_index('TFD')
    dtf_asint = document_term_frequency.astype(int)
    coocc = dtf_asint.T.dot(dtf_asint)

    return document_term_frequency, dtf_asint, coocc

### Calculating Relationship Score: S(i, j)

In [7]:
def get_relationship_between_concepts(concept_1, concept_2, document_term_frequency):
    concept_1_index= document_term_frequency.columns.get_loc(concept_1)
    concept_2_index= document_term_frequency.columns.get_loc(concept_2)
    
    return coocc.iloc[concept_1_index, concept_2_index]

### Significance of a concept in a document: \lambda(c, i)

In [8]:
def get_significance_score(concept, index, document, document_term_frequency, dtf_asint, coocc):
    if(document == None): return 0
    concept_index = document_term_frequency.columns.get_loc(concept)
    freq = dtf_asint.iloc[index[document]][concept_index]
    coocc_row = coocc.iloc[concept_index,:] 
    r = np.array(coocc_row)
    if(sum(r) == 0): return freq
    return (freq)+np.count_nonzero(r)

### Key Sections k_c

In [9]:
def get_doc_to_concepts_list(content, index, top_n, document_term_frequency, dtf_asint, coocc):
    doc_to_concept_list = {}
    relevant_concepts_to_sequence = set()
    
    for each_document in content.keys():
        rt = []
        rc = []
        for each_concept in kg_labels:
            s = get_significance_score(each_concept, index,each_document, document_term_frequency, dtf_asint, coocc)
            if(s <= 0): continue
            if("NodeType" not in kg.node[each_concept]):
                continue
            elif(kg.node[each_concept]["NodeType"] == "ConceptNode"):
                rc.append((each_concept, s))
            elif(kg.node[each_concept]["NodeType"] == "TopicNode"):
                rt.append((each_concept, s))
        
        rt.sort(key=lambda x:x[1])
        rt = rt[::-1]
        
        rc.sort(key=lambda x:x[1])
        rc = rc[::-1]
        
        key_concepts = []
        while(len(rc) and len(key_concepts) < top_n):
            key_concepts.append(rc[0][0])
            relevant_concepts_to_sequence.add(rc[0][0])
            rc.pop(0)
            
        while(len(rt) and len(key_concepts) < top_n):
            key_concepts.append(rt[0][0])
            relevant_concepts_to_sequence.add(rt[0][0])
            rt.pop(0)
            
        for each in rt:
            relevant_concepts_to_sequence.add(each[0])
        
        for each in rc:
            relevant_concepts_to_sequence.add(each[0])
        doc_to_concept_list[each_document] = key_concepts
    return doc_to_concept_list, relevant_concepts_to_sequence

In [10]:
def get_relevant_concepts_for_lp(doc_to_concepts_list):
    rel = []
    for key,val in doc_to_concepts_list.items():
        for each in val:
            rel.append(each)
    return rel   

### Comprehension Burden

In [11]:
def f_cb(sig_score, key_sig_score, relationship):
    return sig_score+key_sig_score

def get_related_concepts(document, index, document_term_frequency):
    concepts = []
    a = np.array(document_term_frequency.iloc[index[document]])
    z = a.nonzero()
    if(len(z[0]) == 0): return []
    for x in np.nditer(z[0]):
        concepts.append(document_term_frequency.columns[x])   
    return concepts
    
def get_cb_document(document, dc, visited, relevant, 
                    document_term_frequency, dtf_asint, coocc, index):
    document_burden = 0.0
    ds = get_related_concepts(document, index, document_term_frequency)
    for d in ds:
        burden = 0.0
        count = 0
        for c in dc:
            if(get_relationship_between_concepts(d, c, document_term_frequency) > 0):
                count += 1
                if(d not in visited):
                    burden += get_significance_score(c, index, document, document_term_frequency, dtf_asint, coocc)
                else:
                    count += 1
        if(count > 0):
            document_burden += burden/count
    return document_burden
    
    
    

### Sequence Generation

In [12]:
def get_linear(nodes):
    parents = []
    for each in nodes:
        if(kg.nodes[each]["NodeType"] == "TopicNode"):
            parents.append(each)
    
    linear = []
    for p in parents:
        linear.append(p)
        children = kg.neighbors(p)
        for c in children:
            if c in nodes and kg.nodes[c]["NodeType"] == "ConceptNode":
                linear.append(c)
    
    for each in nodes:
        if each not in linear:
            linear.append(each)
    
    return linear

def get_weighted_sequences(nodes):
    parents = []
    for each in nodes:
        if(kg.nodes[each]["NodeType"] == "TopicNode"):
            parents.append(each)
            
    weighted = []
    for p in parents:
        weighted.append(p)
        children = kg.neighbors(p)
        all_c = []
        
        for c in children:
            if(c not in nodes): continue
            if("weight" in kg[p][c]):
                all_c.append((c, kg[p][c]["weight"]))
            else:
                all_c.append((c, 0.0))

        all_c.sort(key=lambda x:x[1])
        all_c = all_c[::-1]

        for e in all_c: weighted.append(e[0])
    return weighted
            
    
def get_sequences(nodes):
        linear = get_linear(nodes)
        top_down = linear[::-1]
        weighted = get_weighted_sequences(nodes)
        return linear, top_down, weighted

In [13]:
def get_concepts_to_document_list(doc_to_concepts_list, relevant_concepts):
    concept_to_document_list = {}
    for each_concept in relevant_concepts:
        concept_to_document_list[each_concept] = []
        for doc, kcs in doc_to_concepts_list.items():
            if(each_concept in kcs): 
                concept_to_document_list[each_concept].append(doc)
    return concept_to_document_list
                

In [14]:
def get_burden_for_sequence(sequence, doc_to_concepts_list, 
                            concepts_to_document_list, 
                            document_term_frequency, dtf_asint, coocc, index):
    collective_burden = 0.0
    docs_sequence = []
    
    for each_con in sequence:
        docs_ass = concepts_to_document_list[each_con]
        for each in docs_ass:
            if(each not in docs_sequence):
                docs_sequence.append(each)
    
    visited = set()
    for each_doc in docs_sequence:
        for each_ass_conc in doc_to_concepts_list[each_doc]:
            visited.add(each_ass_conc)

        collective_burden += get_cb_document(each_doc, doc_to_concepts_list[each_doc], 
                                                     visited, sequence, 
                                                     document_term_frequency, dtf_asint, coocc, index)
        
        
    return collective_burden    

In [15]:
def get_score(content, index, top_n, document_term_frequency, dtf_asint, coocc):
    doc_to_concepts_list, relevant_concepts_to_sequence = get_doc_to_concepts_list(content, index, top_n, document_term_frequency, dtf_asint, coocc)
    concepts_to_document_list = get_concepts_to_document_list(doc_to_concepts_list, relevant_concepts_to_sequence)
    
    linear, bottom_up, weighted = get_sequences(relevant_concepts_to_sequence)
    s = get_burden_for_sequence(weighted, doc_to_concepts_list, concepts_to_document_list, 
                                document_term_frequency, dtf_asint, coocc, index)/len(doc_to_concepts_list)
    return (s, doc_to_concepts_list)

In [16]:
def get_required(url):
    content, index, es_order = get_content(url)
    document_term_frequency, dtf_asint, coocc = get_matrices(content, index)
    return content, document_term_frequency, dtf_asint, coocc, index

In [17]:
content, document_term_frequency, dtf_asint, coocc, index = get_required("lps/engage/user_study_clustering_engage.txt")


In [18]:
for i in range(1, 4, 1):
    s, doc_to_concepts_list = get_score(content, index, i, document_term_frequency, dtf_asint, coocc)
    print(i, s)

(1, 33.63333333333333)
(2, 29.166666666666668)
(3, 23.11111111111111)


In [18]:
import os
for root, dirs, files in os.walk("lps/engage/"):  
    for filename in files:
        content, document_term_frequency, dtf_asint, coocc, index = get_required("lps/engage/"+filename)
        print(filename)
        for i in range(1, 4, 1):
            s, doc_to_concepts_list = get_score(content, index, i, document_term_frequency, dtf_asint, coocc)
            print(i, s)
        print("\n")

user_study_clustering_engage.txt
(1, 33.63333333333333)
(2, 29.166666666666668)
(3, 23.81111111111111)


user_study_divide_conquer_engage.txt
(1, 0.0)
(2, 1101.8125)
(3, 1071.3333333333335)


user_study_dynamic_programming_engage.txt
(1, 14.5)
(2, 315.80357142857144)
(3, 305.7738095238095)


user_study_graph_theory_engage.txt
(1, 327.125)
(2, 308.78125)
(3, 293.90625)


user_study_mixture_models_engage.txt
(1, 34.888888888888886)
(2, 29.97222222222222)
(3, 25.685185185185183)


user_study_processes_engage.txt
(1, 0.0)
(2, 11.923076923076923)
(3, 10.384615384615385)


user_study_sorting_algorithms_engage.txt
(1, 140.69230769230768)
(2, 128.30769230769232)
(3, 124.12820512820514)


user_study_supervised_learning.txt
(1, 427.44444444444446)
(2, 410.1111111111111)
(3, 392.77777777777777)


user_study_unsupervised_engage.txt
(1, 275.0)
(2, 258.5)
(3, 242.0)




In [20]:
25.266666666666666
24.5

24.5